In [ ]:
from baseline_LSTM_POS import *
import copy

In [ ]:
sentences, pos_tags = load_data("UD_French-Sequoia/fr_sequoia-ud-train.conllu")

In [ ]:
word_counts = Counter(word for sentence in sentences for word in sentence)
word_to_ix = {word: i+1 for i, word in enumerate(word_counts)}  # +1 pour le padding
word_to_ix['<PAD>'] = 0
word_to_ix['<OOV>'] = len(word_to_ix)

tag_counts = Counter(tag for tags in pos_tags for tag in tags)
tag_to_ix = {tag: i for i, tag in enumerate(tag_counts)}

In [ ]:
embedding_dim = 64
hidden_dim = 128
epochs=50
batch_size=16

In [ ]:
dataset = POSDataset(sentences, pos_tags, word_to_ix, tag_to_ix)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

model = POSModel(len(word_to_ix), embedding_dim, hidden_dim, len(tag_to_ix))
loss_function = nn.CrossEntropyLoss(ignore_index=-1)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

<h2> Training

In [ ]:
for epoch in range(epochs):
    total_loss = 0
    for sentence_in, targets in data_loader:
        model.zero_grad()
        tag_scores = model(sentence_in)
        loss = loss_function(tag_scores.view(-1, len(tag_to_ix)), targets.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(data_loader)}")

In [ ]:
loss, accuracy, precision, recall, f1 = evaluate_model(model, data_loader, loss_function, tag_to_ix)
print(f"Train Accuracy : {accuracy:.4f}")
print(f"Train loss : {loss:.4f}")
print(f"Train precision : {precision:.4f}")
print(f"Train recall : {recall:.4f}")
print(f"Train F1 score : {f1:.4f}")

<h2> Testing

In [ ]:
test_sentences, test_pos_tags = load_data("UD_French-Sequoia/fr_sequoia-ud-test.conllu")
test_dataset = POSDataset(test_sentences, test_pos_tags, word_to_ix, tag_to_ix)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [ ]:
loss, accuracy, precision, recall, f1 = evaluate_model(model, test_data_loader, loss_function, tag_to_ix)
print(f"Test Accuracy : {accuracy:.4f}")
print(f"Test loss : {loss:.4f}")
print(f"Test precision : {precision:.4f}")
print(f"Test recall : {recall:.4f}")
print(f"Test F1 score : {f1:.4f}")